In [1]:
# === Imports ===
import numpy as np
import holoviews as hv
from holoviews import HLine, VLine, Text

from ez_data import *
from pyPulses.utils import (SweepMeasureProductConfig, 
                            sweepMeasureProduct,
                            SweepMeasureParallelepipedConfig, 
                            sweepMeasureParallelepiped)


If you wish to use GPIB instruments, make sure NI488.2 is installed.


In [2]:
# === Measurement Setup ===

path = "data/example_dset.zarr"
ds = MeasurementDataset(path)

swept_names = ("Vtg", "Vbg")
measured_names = ("resistance",)

def dummy_set(val): pass
def dummy_get(): return np.random.exponential(scale=5e3)

# === Transform Function: (Vtg, Vbg) → (n, D) ===

def compute_n_and_D(coords):
    Vtg, Vbg = coords["Vtg"], coords["Vbg"]
    n = 1e12 * (Vtg + Vbg)
    D = 0.1 * (Vtg - Vbg)
    return {"n": n, "D": D}

# === Callback with Metadata + Transform ===
coord_attrs = {
    "Vtg": {"units": "V", "long_name": "Top Gate Voltage"},
    "Vbg": {"units": "V", "long_name": "Bottom Gate Voltage"},
    "n"  : {"units": "cm⁻²", "long_name": "Carrier Density"},
    "D"  : {"units": "V/nm", "long_name": "Displacement Field"}

}
data_attrs = {
    "resistance": {"units": "Ohm", "long_name": "2-terminal resistance"}
}

def pre_callback(index, coords):
    return
    Vtg, Vbg = coords
    n = 1e12 * (Vtg + Vbg)
    D = 0.1 * (Vtg - Vbg)
    print(f"{n:.2e}, {D:.2e}")

def post_callback(index, coords, data):
    coord_dict = dict(zip(swept_names, coords))
    transformed_coords = compute_n_and_D(coord_dict)
    data_dict = dict(zip(measured_names, data))

    full_coords = {**coord_dict, **transformed_coords}
    ds.add_point(full_coords, data_dict, coord_attrs=coord_attrs, data_attrs=data_attrs)

# === Parallelepiped Sweep ===
origin = [5.0, 5.0]
endpoints = np.array([[6.0, 6.0],
                      [6.0,  4.0]])
shape = [21, 11]

para_config = SweepMeasureParallelepipedConfig(
    setters=(dummy_set, dummy_set),
    getters=(dummy_get,),
    swept_name=swept_names,
    measured_name=measured_names,
    time_per_point=0.01,
    origin=origin,
    endpoints=endpoints,
    shape=shape,
    pre_callback=pre_callback,
    post_callback=post_callback,
    retain_return=False
)

# === Product Sweep ===
axes = (np.linspace(-1, 1, 21), np.linspace(-0.5, 0.5, 11))
prod_config = SweepMeasureProductConfig(
    setters=(dummy_set, dummy_set),
    getters=(dummy_get,),
    swept_name=swept_names,
    measured_name=measured_names,
    time_per_point=0.001,
    pre_callback=pre_callback,
    post_callback=post_callback,
    retain_return=False,
    axes=axes
)

In [3]:
# === Run Sweeps ===
print("Running Parallelepiped Sweep")
sweepMeasureParallelepiped(para_config)
# print("Running Product Sweep")
# sweepMeasureProduct(prod_config)
ds.flush_to_zarr() # flush any unsaved data


Running Parallelepiped Sweep


In [4]:
# === Load and Plot Dataset in nD Space ===
ds.load()
# configure_plotting_theme()
plot = plot_2d_data(
    ds.ds, xs = "n", ys = "D", zs = "resistance",
    axes = {'n': np.linspace(1.0e13, 1.2e13, 21), 
            'D': np.linspace(0.0, 0.2, 11)},
    plot_kwargs = {
        'clim': (1e2, 1e5),
        'logz': True,
        'cmap': "plasma",
        'colorbar': True
    },
    plot_mode = 'image'
)
plot

:Image   [x,y]   (resistance)

In [5]:
# == Editing Plot ===
plot = plot.opts(
    title   = "Resistance Map",
    width   = 600, height=500,
    colorbar= True
)
plot

:Image   [x,y]   (resistance)

In [6]:
# === Annotate with Overlays ===
hline = HLine(0).opts(color="red", line_dash="dashed")
vline = VLine(0).opts(color="blue", line_dash="dotted")
label = Text(0, 0, "Charge neutrality").opts(text_align="left", text_baseline="bottom")

full_plot = (plot * hline * vline * label).opts(title="Annotated Map")
full_plot

:Overlay
   .Image.I :Image   [x,y]   (resistance)
   .HLine.I :HLine   [x,y]
   .VLine.I :VLine   [x,y]
   .Text.I  :Text   [x,y]

In [7]:
# === Saving Plots ===
hv.save(full_plot, "data/resistance_map.html")  # interactive HTML

In [8]:
# If we want to save to other formats using a matplotlib backend, we need to 
# recreate the plot so that the colormapping logic stays consistent
hv.extension("matplotlib")
plot = plot_2d_data(
    ds.ds, xs = "n", ys = "D", zs = "resistance",
    axes = {'n': np.linspace(1.0e13, 1.2e13, 21), 
            'D': np.linspace(0.0, 0.2, 11)},
    plot_kwargs = {
        'clim': (1e2, 1e5),
        'logz': True,
        'cmap': "plasma",
        'colorbar': True
    },
    plot_mode = 'image'
)
plot = plot.opts(
    title="Resistance Map",
    fig_inches = (4, 4)
)
hline = HLine(0).opts(color="red", linestyle = "dashed")
vline = VLine(0).opts(color="blue", linestyle = "dotted")
label = Text(0, 0, "Charge neutrality")
full_plot = (plot * hline * vline * label).opts(title="Annotated Map")

hv.save(full_plot, "data/resistance_map.svg", fmt = 'svg')    # vector format
hv.save(full_plot, "data/resistance_map.png", fmt = 'png')    # vector format

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W